In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip3 install pyvi

     |████████████████████████████████| 8.5MB 1.4MB/s 
     |████████████████████████████████| 747kB 34.5MB/s 


In [3]:
from pyvi import ViTokenizer, ViPosTagger

In [4]:
data = pd.read_csv('./reviews.csv')
data

,Review,Label
0,Ở đây nổi tiếng caramen nhưng mà mình lại biế...,1.0
1,Bánh mì sốt vang Thái Thịnh nổi nay mới được ...,-1.0
2,Nghe tên quán đã lâu hôm nay mới có dịp đến ă...,1.0
3,Mua vé online các sự kiện tổ chức ở,1.0
4,chăng so không gian cây xanh ban cây ai cây v...,1.0
...,...,...
5909,Hôm mùng tết có đi qa đây mua cốc trà Topping...,-1.0
5910,Trà sữa socola ngon nhất trong những quán đã ...,1.0
5911,Quán bé do quán vắng mình chó đến ngồi ngoài ...,-1.0
5912,Hôm nay lần đầu đến quán cảm thấy ăn cũng nhâ...,-1.0


In [5]:
data.Label.value_counts()

 1.0    2779
-1.0    2416
 0.0     715
Name: Label, dtype: int64

In [6]:
data.Label.isna().sum()

4

In [7]:
data.fillna(0., inplace = True)

In [11]:
X = data["Review"]
y = data["Label"]
X

0        Ở đây nổi tiếng caramen nhưng mà mình lại biế...
1        Bánh mì sốt vang Thái Thịnh nổi nay mới được ...
2        Nghe tên quán đã lâu hôm nay mới có dịp đến ă...
3                     Mua vé online các sự kiện tổ chức ở
4        chăng so không gian cây xanh ban cây ai cây v...
                              ...                        
5909     Hôm mùng tết có đi qa đây mua cốc trà Topping...
5910     Trà sữa socola ngon nhất trong những quán đã ...
5911     Quán bé do quán vắng mình chó đến ngồi ngoài ...
5912     Hôm nay lần đầu đến quán cảm thấy ăn cũng nhâ...
5913     Chị chủ và anh chủ siêu siêu dễ Menu các món ...
Name: Review, Length: 5914, dtype: object

In [12]:
englishwords = pd.read_json("./englishwords.json")
englishwords.set_index("english", inplace = True)
shortform = pd.read_json("./shortform.json")
shortform.set_index("short", inplace = True)
englishwords

,vietnamese
english,
fun,vui
phone,điện thoại
funny,hay ho
concept,ý tưởng
idea,ý tưởng
smartphone,điện thoại thông minh
no,không
yes,có
capable,có thể


In [13]:
# Loại bỏ kí tự đặc biệt và chuyển thành chữ cái viết thường
import re 

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ' '))
    
    #Loại bỏ các chữ cái lặp nhau
    length = len(text)
    char = 0
    while char <length-1:
        if text[char] == text[char+1]:
            text = text[:char]+text[char+1:]
            #print(text)
            length-=1
            continue
        char+=1
    #loại bỏ số  và các chữ cái lạ:
    digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ(?::|;|=)(?:-)?(?:\)|\(|D|P)_ '
    text = ''.join([i for i in text if i in digits_and_characters])

    text_split = text.split()
    #print(text_split)
    for i in range(len(text_split)):
      t = text_split[i]
      if t in englishwords.index:
        text_split[i] = str(englishwords.loc[t, "vietnamese"])
      elif t in shortform.index:
        text_split[i] = str(shortform.loc[t,"long"]) 
    #print(text_split)
    text = ' '.join(text_split)
    return text

In [17]:
# get rid of special characters and convert to lower case
for i in range (X.shape[0]):
    X.loc[i] = preprocessor(X.loc[i])
X.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


0    ở đây nổi tiếng caramen nhưng mà mình lại biết...
1    bánh mì sốt vang thái thịnh nổi nay mới được đ...
2    nghe tên quán đã lâu hôm nay mới có dịp đến ăn...
3                  mua vé online các sự kiện tổ chức ở
4    chăng so không gian cây xanh ban cây ai cây vi...
5          trà sữa ngon nhân viên bán hàng nhiệt thích
6    không thể mê nổi thái độ của nhân viên từ đầu ...
7    đây là trong những quán ăn mà mình rất vị trí ...
8    ngoài món chính cơm niêu xá xíu và bò tiêu đen...
9    nếu đi từ hàng bồ vào cẩn thận các bạn sẽ nhầm...
Name: Review, dtype: object

In [19]:
X.loc[10]

'thái độ nhân viên phục vụ không mê mà trước giờ trừ gv ra thì mình chưa mê được thái độ nhân viên pg ở đâu nhìn khách như kiểu mày muốn gì mày nhanh nhanh nói với khách cũng với kiểu khách là pet còn các bạn ấy là xin lạy của em một lạy không gian học bài ở đây không ồn mỗi tội ít chỗ quá nên lần nào mình đến cũng không tìm được chỗ ngồi đồ uống bình được cái mình thích ăn thạch ở mát'

In [22]:
ViPosTagger.postagging(ViTokenizer.tokenize(X.loc[10]))

(['thái_độ',
  'nhân_viên',
  'phục_vụ',
  'không',
  'mê',
  'mà',
  'trước',
  'giờ',
  'trừ',
  'gv',
  'ra',
  'thì',
  'mình',
  'chưa',
  'mê',
  'được',
  'thái_độ',
  'nhân_viên',
  'pg',
  'ở',
  'đâu',
  'nhìn',
  'khách',
  'như',
  'kiểu',
  'mày',
  'muốn',
  'gì',
  'mày',
  'nhanh',
  'nhanh',
  'nói',
  'với',
  'khách',
  'cũng',
  'với',
  'kiểu',
  'khách',
  'là',
  'pet',
  'còn',
  'các',
  'bạn',
  'ấy',
  'là',
  'xin',
  'lạy',
  'của',
  'em',
  'một',
  'lạy',
  'không_gian',
  'học',
  'bài',
  'ở',
  'đây',
  'không',
  'ồn',
  'mỗi',
  'tội',
  'ít',
  'chỗ',
  'quá',
  'nên',
  'lần',
  'nào',
  'mình',
  'đến',
  'cũng',
  'không',
  'tìm',
  'được',
  'chỗ',
  'ngồi',
  'đồ',
  'uống',
  'bình',
  'được',
  'cái',
  'mình',
  'thích',
  'ăn',
  'thạch',
  'ở',
  'mát'],
 ['V',
  'N',
  'V',
  'R',
  'V',
  'C',
  'E',
  'N',
  'V',
  'N',
  'V',
  'C',
  'P',
  'R',
  'V',
  'R',
  'V',
  'N',
  'V',
  'E',
  'P',
  'V',
  'N',
  'C',
  'N',
  'P',
  'V

In [58]:
vanva = set()
pos_to_consider = ('A','R', 'S','T', 'V')

In [37]:
def tokenizer(text):
  text = ViTokenizer.tokenize(text)
  return text
print(tokenizer("một bí mật"))

một bí_mật


In [55]:
def getVanva(text):
  postag = ViPosTagger.postagging(text)
  #print(postag)
  for i in range (len(postag[1])):
    if postag[1][i] in pos_to_consider:
      vanva.add(postag[0][i])
  #print(vanva)
  return vanva 

In [56]:
getVanva("một con chó")

set()

In [57]:
getVanva("đẹp tuyệt vời")

{'tuyệt', 'đẹp'}

In [ ]:
getVanva("tuyệt vời quá")

In [42]:
getVanva("quá đẹp_trai")

(['quá', 'đẹp_trai'], ['R', 'A'])
{'đẹp', 'tuyệt', 'quá', 'đẹp_trai'}


{'quá', 'tuyệt', 'đẹp', 'đẹp_trai'}

In [59]:
for i in range (X.shape[0]):
  vanva = getVanva(tokenizer(preprocessor(X.loc[i])))
print(vanva)


{'bụp', 'nghi_ngút', 'khinh_miệt', 'hãy', 'trôi', 'phụ_thuộc', 'fealing', 'mushrom', 'thông_cảm', 'lẽ_ra', 'mắm_nêm', 'sĩ', 'nhát', 'bận_tâm', 'hàu', 'churo', 'từ_chối', 'bán_nước', 'vừa_ý', 'bánh_trái', 'túm_tụm', 'rần_rần', 'ạ', 'vụng', 'gà_tây_viên', 'hô_hào', 'mine', 'sành', 'cty', 'bắn_mỡ', 'bó', 'sốt_me', 'mng', 'nhju', 'olive', 'chuyên_hóa', 'căng', 'impatiently', 'mùi_vị_thế', 'cao_cấp', 'tìn', 'abc', 'ê', 'ngừng_nghỉ', 'thạnh', 'mâm_xôi', 'canot', 'nhần', 'nói_chung', 'cắn_răng', 'lót_dạ', 'ngọt_cốc', 'nạc', 'húng', 'chịu_khó', 'xfim', 'phiên_âm', 'chìm_ngập', 'nhạy', 'liên_lạc', 'lọc_cọc', 'any', 'trôg', 'bắn', 'nhỏgiot', 'gạt_tàn', 'tn', 'gui', 'có_học', 'cho_cùng', 'gia_giảm', 'khay', 'velved', 'cắt', 'vui_tươi', 'không_khéo', 'lợi_dụng', 'ngoại_giao_tiếp', 'vị_giác', 'thật_thà', 'phấp', 'chen_chúc', 'miệng_ăn', 'chewy', 'vẻn_vẹn', 'out', 'hợp_lí', 'hủ', 'mag', 'desnd', 'xem_xét', 'đả_động', 'ngấy', 'choai_choai', 'season', 'chắc_ăn_đứt', 'trình', 'cua_bể', 'thuwong', 'chào

In [50]:
print(len(vanva))

8091


In [52]:
vanva_bag = sorted(vanva)
print(vanva_bag) 

['_hôm', '_mình', 'a', 'abc', 'abd', 'about', 'ac', 'ace', 'acountant', 'acoustic', 'ad', 'ading', 'admin', 'adres', 'advertise', 'ae', 'aen', 'aeon', 'afogato', 'after', 'afternon', 'again', 'ahihi', 'ahip', 'ahuhu', 'ai_bảo', 'ai_ngờ', 'ai_đời', 'ak', 'al', 'alacarte', 'alarcarte', 'ale', 'alfrseco', 'alike', 'alisa', 'alisan', 'almaz', 'alo', 'alone', 'along', 'also', 'always', 'am_hiểu', 'amazed', 'amazing', 'amounts', 'ampid', 'ample', 'an', 'an_di', 'an_toàn', 'an_tâm', 'an_ủi', 'and', 'anh_chị', 'anh_em_em', 'anh_vũ', 'any', 'aots', 'apertizer', 'aple', 'are', 'area', 'aroi', 'around', 'arất', 'as', 'asian', 'asked', 'at', 'atmosphere', 'australia', 'authentic', 'away', 'ba_chỉ', 'ba_rọi', 'baby', 'bac', 'back', 'bacon', 'bad', 'baked', 'bakery', 'bal', 'bami', 'ban_công', 'ban_đầu_vào', 'banana', 'banc', 'banchan', 'bang', 'bao', 'bao_bì', 'bao_cấp', 'bao_giờ', 'bao_gồm', 'bao_lâu', 'bao_lâu_nay', 'bao_nhiêu', 'bao_quát', 'bao_trùm', 'bao_tải', 'bao_tử', 'bao_xa_vậy', 'baoh', '

In [ ]:
for text in range(X.shape[0]):
  